# Medium Clapping – Setup
Automated clapping for medium articles

There are four parts to the setup
1. Installing the appropriate python libraries. Most software is standard except the Google API and selenium
2. Setting up an appropriate config file
3. Installing chromedriver, an automatable web browser
4. Log into medium

To Do: updated information on how to get an API key

## Part 1: Python Packages
You need to be able to run the following imports. Run conda or pip install at your terminal to install the software. Alterntively, run the following cell

In [ ]:
# Warning: This cell takes a while
%conda install more-itertools selenium google-api-python-client pandas numpy

In [5]:
# Standard Imports
import re
import json
import sys, os, shutil
from   pathlib import Path
import time

import itertools
import more_itertools  # conda install more-itertools

import pandas
import numpy

import selenium

In [6]:
# This is optional if you don't intend to get links from a google sheet
import googleapiclient # conda install google-api-python-client

In [7]:
from utilities import load_config, write_config, profile_path, get_browser

## Part 2: Set up a config file

This will be a simple json file containing variables
- api_key

One of:
- (a) spreadsheet_id: the id of a public google sheet to scrape for links; or
- (b) medium_urls:    the path to a file containing urls (see below)

And
- chromedriver_location (set in part 3)

If you don't want to use a google sheet, you can make a list of urls in a local file. If so:
medium_urls should be a string with the relative path (i.e. from the project diretory) or an absolute path to a text file. The text file should be a list of URLs, each on their own line. Alternatively, the file could be a spreadsheet as a csv or xls(x).

This notebook will detect if a file exists and if not make a new one.
- If you want to delete the file and remake it, run make_new_config() in this notebook.
- If you want to edit the file, you can run load_config(), edit the resulting dictionary, and then write_config(new_dict)

Note that no checks on the validity of the configuration info are made

In [3]:
def get_api_key():
    return input('Please enter a Google API key:\n\n')

In [4]:
def get_url_resource():
    spreadsheet    = input('\n\nPlease enter either\n'
                       '  (a) the URL of a public Google Sheet to scrape for links; or \n'
                       '  (b) The path to a text, csv, or excel file containing links \n\n')

    # Check to see if the input is a URL or else assume it is the path to a file
    #  If it is a file path, just return the path
    if not re.match('http.*', spreadsheet):
        print('Assuming this is a file path')
        return 'medium_urls', spreadsheet
    
    # Parse the spreadsheet id from the URL
    #  Regex is from the Google API documentation
    spreadsheet_re = '/spreadsheets/d/([a-zA-Z0-9-_]+)'
    spreadsheet_id = re.search(spreadsheet_re, spreadsheet)
    
    if not spreadsheet_id:
        raise ValueError('''That doesn't appear to be a Google Sheet or file path''')
        
    return 'spreadsheet_id', spreadsheet_id[1]

In [5]:
def make_new_config():
    config = {}
    config['api_key'] = get_api_key()
    
    key, val = get_url_resource()
    config[key] = val
    
    write_config(config)
    
    print('Configuration File Succesfully Built')

In [6]:
def yes_no_prompt():
    to_delete = input('''Enter y for yes, or n or blank for no:  ''')
    return to_delete in ['y', 'Y', 'yes', 'Yes']

In [7]:
try:
    load_config()
    print('Configuration file is detected. You may proceed to step 3')
except utilities.FileNotFound:
    print('No config file detected, creating a new one one')
    make_new_config()
except utilities.CorruptConfiguration:
    print('''Corrupted config file detected, please confirm to delete and make a new one.''')
    if yes_no_prompt():
        print('Okay, deleting the file and proceeding to setup a new one')
        make_new_config()
    else:
        print('Terminating with failure.')

Configuration file is detected. You may proceed to step 3


## Part 3: Chomedriver
This will look for chromedriver, a webbrowser

You should start by downloading that here
https://chromedriver.chromium.org/

By default the browser should be in: /Library/Application Support/Google/chromedriver

You could also put in the project directory

If the file is found in your downloads folder, you will be prompted to move it to default directory

In [8]:
try:
    config = load_config()
except:
    print('Error loading the config file... you need to go back to step 2')

In [9]:
chrome    = Path.home() / Path('./Library/Application Support/Google/')
here      = Path('.')
downloads = Path.home() / Path('./Downloads')

default_name = Path('./chromedriver')
file_name    = default_name

In [10]:
chromedriver = None

# Look in the default places
for path in [chrome, here, downloads]:
    if (path/file_name).exists():
        display = path if path != here else 'this directory'
        print(f'Found chrome driver in {display}')
        chromedriver = path/file_name
        break
        
# Prompt the user if chrome driver hasn't been found
if not chromedriver:
    file_path   = input('What directory is your chromedriver in? Empty input to terminate.\n'
                        'You can download it here: https://chromedriver.chromium.org/ and come back')
    if file_path:
        file_name_2 = input('What is the filename? Empty to default to chromedriver')
        if file_name_2:
            file_name = file_name_2
        if (file_path/file_name).exists():
            chromedriver = file_path/file_name
        else:
            print('''Sorry can't find the file''')
            
if chromedriver:
    # Prompt the user to move chromedriver to a reasonable location
    if chromedriver not in [chrome/file_name, here/file_name]:
        print(f'Would you like to move this to {chrome/default_name}?')
        move = yes_no_prompt()
        if move:
            start  = str(chromedriver)
            dest   = str(chrome/default_name)
            os.rename(source, dest)
            
    # Update the config file
    config['chromedriver_location'] = str(chromedriver)
    write_config(config)
    
    print('Congratulations, you can move to step 4')
else:
    print('Terminating because you need to download chromedriver')

Found chrome driver in /Users/rcharan/Library/Application Support/Google
Congratulations, you can move to step 4


# Part 4: Logging into Medium

This part will create a user profile for the web browser chromedriver. After you log into medium, it will remember your credentials in the same way as you may be used to in a standard web browser.

If you don't want to do this, you will have to log into medium every time.

Re-run the 

In [11]:
config = load_config(require_chrome = True)

In [13]:
print('''Do you want to create a user profile in chromedriver so that you don't have to log in all the time?\n\n'''
      '''If you decline, the configuration file will be updated to not use user profiles.\n'''
      ''' -  You can change this later by re-running this cell to enable or disable the profile\n'''
      ''' -  On disable, you will have the choice to delete an existing profile if it exists\n''')

have_profile = yes_no_prompt()

# Update the configuration file
print(f'\nConfiguration profile is updated to {"enable" if have_profile else "disable"} a user profile')
config['have_profile'] = have_profile
write_config(config)

# If the profile was declined, prompt to delete existing profiles
if not have_profile and profile_path.exists():
    print('\n\nDo you want to delete the existing profile\n\n')
    remove = yes_no_prompt()
                    
    if remove:
        shutil.rmtree('./chromedriver-profile')

Do you want to create a user profile in chromedriver so that you don't have to log in all the time?

If you decline, the configuration file will be updated to not use user profiles.
 -  You can change this later by re-running this cell to enable or disable the profile
 -  On disable, you will have the choice to delete the existing profile

Enter y for yes, or n or blank for no:  y

Configuration profile is updated to enable a user profile


_*Please log into medium!*_ Then come back here and close the browser from the notebook

In [14]:
browser = get_browser()
if browser:
    browser.get('https://medium.com/')

In [15]:
if browser:
    browser.close()
    print('Congratulations you are all set up!')

Congratulations you are all set up!


Please note that the best way to close chromedriver is by
- Using browser.close(); or 
- Quitting the application (*not* just closing the window)

If you close the window without quitting the application, it will lock the user profile and lead to later errors